In [ ]:

# !apt install chromium-chromedriver
!pip install selenium
!pip install tabulate

In [205]:
import pandas as  pd 
import numpy as np
import logging
logging.basicConfig(filename='app.log', filemode='w', format='%(name)s - %(message)s',level=logging.INFO)
import sys

sys.path.append('C:\\Users/peter/Google Drive/Colab Notebooks/Småprojekt/')

import fixa_features as ff
import class_model as mcl

from selenium import webdriver 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException
import time
import sklearn
from sklearn.ensemble import RandomForestClassifier as RF

print("sklearn version", sklearn.__version__)
print("numpy version  ", np.__version__)

sklearn version 0.21.3
numpy version   1.16.5


# Ladda v75-sida med 7 avdelningar/lopp 

In [206]:
def V75_sida(driver,V75page):
    # run firefox webdriver from executable path of your choice

    # get web pages
    data = []
    driver.get(V75page+'avd1')
    but = driver.find_element_by_id('onetrust-accept-btn-handler')
    but.click()
    driver.fullscreen_window()
  
    anp = driver.find_elements_by_css_selector('div.active-game-type-v75 div.startlist-header.game-background-pattern--long button:nth-child(3)')
    anp[0].click()

    cl = driver.find_elements_by_css_selector('div.modal-container  div.modal__body  button.css-tqseha-Button-styles--root-Button--Button')
    cl[0].click()    # Rensa


    cl = driver.find_elements_by_css_selector('ul > li:nth-child(10)')
    cl[1].click()       #Distans och spår

    #cl = driver.find_elements_by_css_selector('ul > li:nth-child(8)')
    #cl[1].click()       #vodds

    #cl = driver.find_elements_by_css_selector('ul > li:nth-child(9)')
    #cl[1].click()       #vodds

    #cl = driver.find_elements_by_css_selector('div.modal-container  div.modal__body #button.css-1fh4n7y-Button-styles--root-PrimaryButton-styles--root-StartlistDisplayOptionsDialog-styles--saveButton-PrimaryButton--PrimaryButton-#StartlistDisplayOptionsDialog-styles--saveButton')
 #   cl[0].click()       #save

    print(V75page)
    print('anpassa')

    return driver


In [207]:
#anp = driver.find_elements_by_css_selector('div.active-game-type-v75 div.startlist-header.game-background-pattern--long button:nth-child(3)')
#print(len(anp))
#anp[0].click()


In [208]:
def getSpår(startnr, avd, i):
    logging.warning('startar getSpår')
    #pth='div:nth-child('+str(avd)+') > div > table > tbody > tr:nth-child('+str(i)+') > td.postPositionAndDistance-col'
    try:    
        spår = startnr.find_element_by_css_selector('.postPositionAndDistance-col')

        txt = spår.text.split()
        spr = None
        dis = None

        if len(txt) > 0:
            dis = txt[0]
    
        if len(txt) == 3:
            spr = txt[2]
    except:
        logging.warning('dist/spår hittades ej')
        spr='ej'
        dis='ej'
    return spr,dis



In [209]:
def getVodds(startnr, avd, häst):
    logging.warning('startar getVodds')

    #pth = '#main > div.active-game-type-v75 > div:nth-child(2) > div > div > div > div > div > div:nth-child(7) > div:nth-child('+str(avd)+') > div > table > tbody > tr:nth-child('+str(häst)+') > td.vOdds-col'
          
    vo = startnr.find_element_by_css_selector('.vOdds-col')

    return vo.text    


In [210]:
def getPodds(startnr, avd, häst):
    logging.warning('startar getPodds')
    #pth = '#main > div.active-game-type-v75 > div:nth-child(2) > div > div > div > div > div > div:nth-child(7) > div:nth-child('+str(avd)+') > div > table > tbody > tr:nth-child('+str(häst)+') > td.pOdds-col'
         
    po = startnr.find_element_by_css_selector('.pOdds-col')
    
    return po.text    


In [211]:
def getHist(avd, häst, startnr):
    logging.warning('startar getHist')
    history=[]
    try:
        for child in range(1,6):
            dist = 'table > tbody > tr:nth-child(' +str(child)+ ') > td.distance-col'
            plac = 'table > tbody > tr:nth-child(' +str(child)+ ') > td.place-col'
            spår = 'table > tbody > tr:nth-child(' +str(child)+ ') > td.position-col'
            odds = 'table > tbody > tr:nth-child(' +str(child)+ ') > td.odds-col'
            try:
                plac = driver.find_element_by_css_selector(plac)
            except:
                logging.warning("hist except - try click history again #########")
                startnr.click()
                plac = driver.find_element_by_css_selector(plac)

            logging.warning('plac '+plac.text)
            dist = driver.find_element_by_css_selector(dist)
            logging.warning('dist '+dist.text)
            spår = driver.find_element_by_css_selector(spår)
            logging.warning('spår '+spår.text)
            odds = driver.find_element_by_css_selector(odds)
            logging.warning('odds '+odds.text)
            history.extend([plac.text, dist.text, spår.text, odds.text])
    except:
        logging.warning("history except")
        return history
    
    logging.warning('History = '+ str(history))
    return history

    datum        20644 non-null object
    bana         20644 non-null object
    avd          20644 non-null int64
    häst         20644 non-null object
    plac         20644 non-null float64
    kön          20644 non-null object
    ålder        20644 non-null int64
    kusk         20644 non-null object
    spår         20644 non-null int32
    vodds        20644 non-null float64
    podds        20644 non-null float64
    start        20644 non-null int64
    lopp_dist    20644 non-null object
    dist         20644 non-null int32
    kr           20644 non-null int32

h1_dat       20644 non-null object
h1_bana      20644 non-null object
h1_kusk      20644 non-null object
h1_plac      20644 non-null int32
h1_dist      20644 non-null int32
h1_spår      20644 non-null int32
h1_kmtid     20643 non-null object
h1_odds      20644 non-null float64
h1_pris      18905 non-null object
h2_dat       20644 non-null object
h2_bana      20644 non-null object
h2_kusk      20644 non-null object
h2_plac      20644 non-null int32
h2_dist      20644 non-null int32
h2_spår      20644 non-null int32
h2_kmtid     20643 non-null object
h2_odds      20644 non-null float64
h2_pris      18706 non-null object
h3_dat       20644 non-null object
h3_bana      20644 non-null object
h3_kusk      20644 non-null object
h3_plac      20644 non-null int32
h3_dist      20644 non-null int32
h3_spår      20644 non-null int32
h3_kmtid     20643 non-null object
h3_odds      20644 non-null float64
h3_pris      18547 non-null object
h4_dat       20644 non-null object
h4_bana      20644 non-null object
h4_kusk      20644 non-null object
h4_plac      20644 non-null int32
h4_dist      20644 non-null int32
h4_spår      20644 non-null int32
h4_kmtid     20643 non-null object
h4_odds      20644 non-null float64
h4_pris      18316 non-null object
h5_dat       20644 non-null object
h5_bana      20644 non-null object
h5_kusk      20644 non-null object
h5_plac      20644 non-null int32
h5_dist      20644 non-null int32
h5_spår      20644 non-null int32
h5_kmtid     20642 non-null object
h5_odds      20644 non-null float64
h5_pris      18107 non-null object

# Läs in avdelning för avdelning och skrapa
öppna varje lopp och gå igenom all hästarna inklusive deras historik

In [212]:
def skrapa(avdelningar, V75page):
    driver.implicitly_wait(10) # seconds
    logging.warning('STARTING skrapa')
    print('starta skrapa')
    
    dat = V75page.split('spel/')[1][0:10]  #gräv ut datum
    komplett=[]
    #driver.fullscreen_window()
    for avd in avdelningar:
        logging.warning('avd '+ str(avd))

        # open avd
        driver.get(V75page+'avd'+str(avd))
        d=driver.find_element_by_css_selector('div > div.cell-wrapper > div.race-combined-info > div.slanted.race-distance-start-method > span')    
        
        split1 = d.text.split()[0][0:4]   # distans (använd hästens distans ist om den finns)
        split2 = d.text.split()[1]        # start

        css='#main .game-view table[data-test-id="startlist-race-'+str(avd)+'"] tbody tr[data-test-id="startlist-row"]'
        logging.warning(css)
        gt = driver.find_elements_by_css_selector(css) 
        antal = len(gt)
        print('antal',antal)
        häst = 0
        for startnr in gt:
            häst = häst+1
            try:
                #print('startnr.text',startnr.text)
                nm = startnr.find_element_by_css_selector('.name-container')
                namn = nm.text
                print(avd, häst, nm.text)
            except:
                logging.warning('avd '+str(avd)+' Startnr: '+str(häst)+'  hittades inte')
                break    

            spår, dist = getSpår(startnr, avd, häst)
            
            if spår.lower()=='ej':
                logging.warning('avd: '+str(avd)+' startnr: '+str(häst)+' struken')
                continue
            else:   
                if dist==None:
                    dist = split1  # använd loppdistans om det saknas hos hästen
                #namn = gt.text
                vodds = getVodds(startnr, avd,häst)
                podds = getPodds(startnr, avd,häst)
                if vodds.lower() == 'ej' or podds.lower() == 'ej':
                    logging.warning('avd: '+str(avd)+' startnr: '+str(häst)+' struken')
                    continue    

                logging.warning('avd: '+str(avd)+' Startnr: '+str(häst)+' '+namn)  

                start = split2

                horse = [dat, avd, namn, dist, start, spår, vodds, podds]
              
                startnr.click()  # open for history
                
                #gh=getHist(avd,häst)
                horse.extend(getHist(avd,häst,startnr))
                komplett.append(horse)
                #print('history:',history)
                startnr.click()   # close

    return komplett

    

# spara lopp data

In [213]:
def spara(df,path, omgsav, avd):
    stravd = ''.join(str(x) for x in avd)
    filnamn=path+omgsav+stravd+'test.csv'
    df.to_csv(filnamn, index=False)

# Bins

In [214]:
def bins(df):
    bins = pd.DataFrame({'fr': [1.0, 2.2, 3.0, 3.8, 4.4, 5.2,
                                6.0, 6.9, 7.7, 8.7, 9.7, 10.8,
                                12.1, 13.6, 15.4, 17.3, 19.1, 21.6,
                                24.4, 27.3, 30.9, 34.8, 39.3, 44.5,
                                50.1, 56.7, 64.5, 75.3, 89.5, 111.7,
                                553.5],
                         'to': [2.2, 3.0, 3.8, 4.4, 5.2,
                                6.0, 6.9, 7.7, 8.7, 9.7, 10.8,
                                12.1, 13.6, 15.4, 17.3, 19.1, 21.6,
                                24.4, 27.3, 30.9, 34.8, 39.3, 44.5,
                                50.1, 56.7, 64.5, 75.3, 89.5, 111.7,
                                553.5, 2000],
                         'bin': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9,
                                 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
                                 20, 21, 22, 23, 24, 25, 26, 27, 28, 29,
                                 30]
                         })

    val = bins.loc[:, 'fr':'to'].apply(tuple, 1).tolist()
    indx = pd.IntervalIndex.from_tuples(val, closed='right')
    dff = df.copy()
    dff['bins'] = bins.loc[indx.get_indexer(dff['vodds']), 'bin'].values

    return dff


# Main - init

In [215]:
#knn1 = model('knn_class1.model',odds_fr=2.8, odds_to=8.7, thresh=1.4,  proba_lim=0.12, med_odds=True)  #vann -426  43-57
#knn2 = model('knn_class2.model',odds_fr=2.4, odds_to=12.6, thresh=1.4, proba_lim=0.4)                  #vann 31    44-56
#Cat1= model('CatB_class1.model',odds_fr=2.3, odds_to=14.6,thresh=1.2,  proba_lim=0.09, med_odds=True)  #vann 29
#Cat2= model('CatB_class2.model',odds_fr=1.4,odds_to=56.7, thresh=1.0,  proba_lim=0.1)                  #vann 1665
#rf1 = model('RF_classLL1.model',odds_fr=2.5,odds_to=20.1, thresh=1.1,  proba_lim=0.2,  med_odds=True)  #vann 1839 80-20 
#rf2 = model('RF_classLL2.model',odds_fr=2.6,odds_to=20.0, thresh=1.1, proba_lim=0.2)                   #vann 1695  62-38 
    
MODELS = {}
MODELS['knn2'] = {'model':mcl.Model('knn_class2.model',odds_fr=1.1,odds_to=50, thresh=1.1, proba_lim=0.1)} #vann 31    44-56
#MODELS['Cat1'] = {'model':mcl.Model('CatB_class1.model',odds_fr=1.1,odds_to=50, thresh=1.1, proba_lim=0.1, med_odds=True)}  #vann 29
#MODELS['Cat2'] = {'model':mcl.Model('CatB_class2.model',odds_fr=1.1,odds_to=50, thresh=1.1, proba_lim=0.1) }                #vann 1665
MODELS['Cat1h'] = {'model':mcl.Model('CatB_class1h.model',odds_fr=1.1,odds_to=50, thresh=1.1, proba_lim=0.1, med_odds=True)}  #vann 747
MODELS['Cat2h'] = {'model':mcl.Model('CatB_class2h.model',odds_fr=1.1,odds_to=50, thresh=1.1, proba_lim=0.1) }                #vann 1665
MODELS['Cat2u'] = {'model':mcl.Model('CatB_class2u.model',odds_fr=1.1,odds_to=50, thresh=1.1, proba_lim=0.1,häst_suf='u')}  #vann 29
# MODELS['rf1'] =  {'model':mcl.Model('RF_classLL1.model',odds_fr=1.1,odds_to=50, thresh=1.1, proba_lim=0.1,med_odds=True)} #vann 1839  80-20 
# MODELS['rf2'] =  {'model':mcl.Model('RF_classLL2.model',odds_fr=1.1,odds_to=50, thresh=1.1, proba_lim=0.1)}             #vann 1695  62-38 
MODELS['rf301'] =  {'model':mcl.Model('rf30_class1.model',odds_fr=1.1,odds_to=50,thresh=1.0,
                                                          proba_lim=0.1,bins=True,med_odds=True)} #vann 1225 82-18 
MODELS['rf302'] =  {'model':mcl.Model('rf30_class2.model',odds_fr=1.1,odds_to=50, thresh=1.0,proba_lim=0.1,bins=True)} #vann 991 61-39 

# MODELS['knn2'] = {'model':mcl.Model('knn_class2.model',odds_fr=2.4, odds_to=12.6, thresh=1.4, proba_lim=0.4)} #vann 31    44-56
# # MODELS['Cat1'] = {'model':mcl.Model('CatB_class1.model',odds_fr=2.3, odds_to=14.6,thresh=1.2,  proba_lim=0.09, med_odds=True)}  #vann 29
# # MODELS['Cat2'] = {'model':mcl.Model('CatB_class2.model',odds_fr=1.4,odds_to=56.7,thresh=1.0,  proba_lim=0.1) }                #vann 1665
# MODELS['Cat1h'] = {'model':mcl.Model('CatB_class1h.model',odds_fr=2.7, odds_to=48.6,thresh=1.4,proba_lim=0.1, med_odds=True)}  #vann 747
# MODELS['Cat2h'] = {'model':mcl.Model('CatB_class2h.model',odds_fr=1.14,odds_to=52.5,thresh=1.79, proba_lim=0.1) }                #vann 1665
# MODELS['Cat2u'] = {'model':mcl.Model('CatB_class2u.model',odds_fr=1.14, odds_to=52.5,thresh=1.79, proba_lim=0.1,häst_suf='u')}  #vann 29
# # MODELS['rf1'] =  {'model':mcl.Model('RF_classLL1.model',odds_fr=2.5,odds_to=20.1,thresh=1.1, proba_lim=0.2,med_odds=True, bins=False)} #vann 1839  80-20 
# # MODELS['rf2'] =  {'model':mcl.Model('RF_classLL2.model',odds_fr=2.6,odds_to=20.0, thresh=1.1, proba_lim=0.2, bins=False)}             #vann 1695  62-38 
# MODELS['rf301'] =  {'model':mcl.Model('rf30_class1.model',odds_fr=1.1,odds_to=50, thresh=1.0,
#                                                          proba_lim=0.1,bins=True,med_odds=True)} #vann 1225 82-18 
# MODELS['rf302'] =  {'model':mcl.Model('rf30_class2.model',odds_fr=1.1,odds_to=50, thresh=1.0,proba_lim=0.1,bins=True)}     #vann 991 61-39 

OMGÅNGAR = {
    1: '2020-04-25/V75/aby',
    2: '2020-05-02/V75/orebro',
    3: '2020-04-18/V75/umaker',
    4: '2020-05-09/V75/aby', 
    5: '2020-05-16/V75/mantorp', 
    6: '2020-05-23/V75/gavle',
    7: '2020-05-30/V75/solvalla',
    8: '2020-06-06/V75/ostersund',
    9: '2020-06-13/V75/boden',
    10: '2020-06-14/V75/bjerke',   
    11 :'2020-07-04/V75/halmstad',   
    12:'2020-06-21/V75/kalmar',   
    13: '2020-07-11/V75/arjang',  
    14: '2020-07-19/V75/axevalla',  
     0: '2020-07-25/V75/bollnas',  
     
} ##### Sätt alltid senaste omgång till 0 och döp om när ny 0:a kommer
  

knn_class2.model odds_fr 1.1 odds_to 50 thresh 1.1 proba_lim 0.1 häst_suf h med_odds False bins True
CatB_class1h.model odds_fr 1.1 odds_to 50 thresh 1.1 proba_lim 0.1 häst_suf h med_odds True bins True
CatB_class2h.model odds_fr 1.1 odds_to 50 thresh 1.1 proba_lim 0.1 häst_suf h med_odds False bins True
CatB_class2u.model odds_fr 1.1 odds_to 50 thresh 1.1 proba_lim 0.1 häst_suf u med_odds False bins True
rf30_class1.model odds_fr 1.1 odds_to 50 thresh 1.0 proba_lim 0.1 häst_suf h med_odds True bins True
rf30_class2.model odds_fr 1.1 odds_to 50 thresh 1.0 proba_lim 0.1 häst_suf h med_odds False bins True


In [216]:

driver = webdriver.Chrome(executable_path='C:\\Users\peter\\Documents\\MyProjects\\gecko\\chromedriver.exe')
#####
driver.implicitly_wait(10) # seconds

logging.warning('STARTING NEW DRIVER')
omg = OMGÅNGAR[0] 
print('omgång:', omg)
omg_sav = omg.replace('/V75/','-')
V75url = 'https://www.atg.se/spel/'+omg

V75_sida(driver, V75url)


omgång: 2020-07-25/V75/bollnas
https://www.atg.se/spel/2020-07-25/V75/bollnas
anpassa


<selenium.webdriver.chrome.webdriver.WebDriver (session="e579661c83e780a5d8797fdfa205036c")>

# Main - web scraping plus predict och spara

In [244]:

if False: 
    omg = OMGÅNGAR[0] 
    print('omgång:', omg)
    omg_sav = omg.replace('/V75/','-')
    V75url = 'https://www.atg.se/spel/'+omg
    
    avd = [1,2,3,4,5,6,7]  ## range(1,8) för alla avdelningar, [2, 4] för 2 och 4

    komplett=skrapa(avd, V75url)  
    df=pd.DataFrame(komplett)
    df.columns=['datum','avd','häst', 'dist', 'start', 'spår', 'vodds', 'podds', 
    'h1_plac', 'h1_dist', 'h1_spår', 'h1_odds',
    'h2_plac', 'h2_dist', 'h2_spår', 'h2_odds',
    'h3_plac', 'h3_dist', 'h3_spår', 'h3_odds',
    'h4_plac', 'h4_dist', 'h4_spår', 'h4_odds',
    'h5_plac', 'h5_dist', 'h5_spår', 'h5_odds']

    df = ff.fix_features(df, True)
    df['vann'] = False

if True:  # kör resultat
    
    df.reset_index(inplace=True,drop=True)

    # add resuls 
    res = df[['datum','avd','häst','vann','vodds']]
    for i,m in MODELS.items():
        MODELS[i]['res'] = MODELS[i]['model'].result(df.drop('vann',axis=1),i)   
        res = pd.concat([res,MODELS[i]['res']], axis=1, sort=False)
    
    #SCALING - lägg in i class model med find('NN')
    #import pickle
    #scaler = pickle.load(open(modellpath+'NN_scaler.model', 'rb'))
    #dfNN = scaler.transform(df.drop(['häst','avd','datum'],axis=1))
    #res_proba = resultat_proba(dfNN, 'NN_class1.model', modellpath)     

    lopp_path='C:\\Users/peter/Documents/MyProjects/PyProj/Trav/'
    spara(df,lopp_path,omg_sav,avd)
    
    print("\nATT SPELA PÅ")
    print(res.loc[res.rf301 | res.rf302 | res.Cat1h | res.Cat2h | res.Cat2u,['avd','häst','vodds','rf301','rf302','Cat1h','Cat2h','Cat2u','knn2']])
    


ATT SPELA PÅ
    avd               häst  vodds  rf301  rf302  Cat1h  Cat2h  Cat2u   knn2
0     1     GUILLAUME BOKO   3.09   True   True  False  False  False  False
1     1  DOCTOR DOXEY ZENZ   8.35   True   True  False  False  False  False
5     1         TJACKO ZAZ  10.85   True  False  False  False  False  False
8     1  HEADING REFERENCE   8.95   True  False  False  False  False   True
10    2          ARABESQUE   7.63   True  False  False  False  False   True
14    2        EKÅS FUTURE   7.61   True   True  False  False  False  False
22    3         QUEER FISH   8.61   True  False  False  False  False   True
27    3    ROSE RUN SYDNEY   7.96   True   True  False  False  False   True
30    4       SPANG VIKTOR  11.73   True  False  False  False  False  False
38    4   KRINGLERS VIKTOR  11.32   True  False  False  False  False   True
44    5   ESTELLE NORDIQUE   5.80   True   True  False  False  False  False
59    6            QUARCIA   3.46   True  False  False  False  False  Fals

In [241]:
res.loc[res.rf301 | res.rf302 | res.Cat1h | res.Cat2h | res.Cat2u,['avd','häst','vodds','rf301','rf302','Cat1h','Cat2h','Cat2u','knn2']].head(60) 

,avd,häst,vodds,rf301,rf302,Cat1h,Cat2h,Cat2u,knn2
0,1,GUILLAUME BOKO,3.09,True,True,False,False,False,False
1,1,DOCTOR DOXEY ZENZ,8.35,True,True,False,False,False,False
5,1,TJACKO ZAZ,10.85,True,False,False,False,False,False
8,1,HEADING REFERENCE,8.95,True,False,False,False,False,True
10,2,ARABESQUE,7.63,True,False,False,False,False,True
14,2,EKÅS FUTURE,7.61,True,True,False,False,False,False
22,3,QUEER FISH,8.61,True,False,False,False,False,True
27,3,ROSE RUN SYDNEY,7.96,True,True,False,False,False,True
30,4,SPANG VIKTOR,11.73,True,False,False,False,False,False
38,4,KRINGLERS VIKTOR,11.32,True,False,False,False,False,True


In [240]:

res.loc[res.rf301 | res.rf302 | res.Cat1h | res.Cat2h | res.Cat2u,['avd','häst','vodds','rf301','rf302','Cat1h','Cat2h','Cat2u','knn2']].tail(24)

,avd,häst,vodds,rf301,rf302,Cat1h,Cat2h,Cat2u,knn2
0,1,GUILLAUME BOKO,3.09,True,True,False,False,False,False
1,1,DOCTOR DOXEY ZENZ,8.35,True,True,False,False,False,False
5,1,TJACKO ZAZ,10.85,True,False,False,False,False,False
8,1,HEADING REFERENCE,8.95,True,False,False,False,False,True
10,2,ARABESQUE,7.63,True,False,False,False,False,True
14,2,EKÅS FUTURE,7.61,True,True,False,False,False,False
22,3,QUEER FISH,8.61,True,False,False,False,False,True
27,3,ROSE RUN SYDNEY,7.96,True,True,False,False,False,True
30,4,SPANG VIKTOR,11.73,True,False,False,False,False,False
38,4,KRINGLERS VIKTOR,11.32,True,False,False,False,False,True


In [242]:

df.loc[df.häst=="RAJA RIBB",['vann']]=True
df.loc[df.häst=="CHAMP LANE",['vann']]=True
df.loc[df.häst=="DREAMMOKO",['vann']]=True
df.loc[df.häst=="RAPPNOR",['vann']]=True
df.loc[df.häst=="CYRANO DE B.",['vann']]=True
df.loc[df.häst=="ANTHARA BI",['vann']]=True
df.loc[df.häst=="VAE VICTIS CLUB",['vann']]=True
df.loc[df.vann,:]


,datum,avd,häst,dist,start,spår,vodds,podds,h1_plac,h1_dist,...,h4_plac,h4_dist,h4_spår,h4_odds,h5_plac,h5_dist,h5_spår,h5_odds,bins,vann
16,2020-07-25,2,CHAMP LANE,2140,1,8,1.92,1.345,1,2140,...,1,2140,8,1.49,2,1640,8,1.26,0,True
21,2020-07-25,3,DREAMMOKO,1640,1,2,3.71,1.535,5,2140,...,2,1640,6,3.63,3,2140,6,11.00,3,True
34,2020-07-25,4,RAPPNOR,2660,1,2,14.10,3.990,13,2180,...,1,1640,1,4.47,1,2160,1,6.57,14,True
49,2020-07-25,5,CYRANO DE B.,3140,1,10,26.32,4.880,3,2080,...,5,2100,6,60.05,5,2640,6,7.00,20,True
57,2020-07-25,6,ANTHARA BI,2160,1,1,3.06,1.570,1,2140,...,1,2160,2,2.09,1,2140,2,8.43,2,True
69,2020-07-25,7,VAE VICTIS CLUB,1640,1,10,4.40,2.000,2,3040,...,14,1650,0,2.20,3,1650,0,2.40,4,True


In [211]:
driver.quit()

# Evaluauera alla modeller one-by-one med Kelly

### Läs in alla nya OMGÅNGAR till df2

In [245]:
avd=[1,2,3,4,5,6,7]
path='C:\\Users/peter/Documents/MyProjects/PyProj/Trav/'
stravd = ''.join(str(x) for x in avd)

df2=pd.DataFrame()
for key,omg in OMGÅNGAR.items():
    # if key==0:
    #     continue
    print(key,omg)
    omg_sav = omg.replace('/V75/','-')
    filnamn=path+omg_sav+stravd+'test.csv'
    tmp = pd.read_csv(filnamn)
    df2 = pd.concat([df2,tmp])

#df2.drop(['datum','avd'],axis=1,inplace=True)    
df2.reset_index(inplace=True,drop=True)
df2.sample(10)


1 2020-04-25/V75/aby
2 2020-05-02/V75/orebro
3 2020-04-18/V75/umaker
4 2020-05-09/V75/aby
5 2020-05-16/V75/mantorp
6 2020-05-23/V75/gavle
7 2020-05-30/V75/solvalla
8 2020-06-06/V75/ostersund
9 2020-06-13/V75/boden
10 2020-06-14/V75/bjerke
11 2020-07-04/V75/halmstad
12 2020-06-21/V75/kalmar
13 2020-07-11/V75/arjang
14 2020-07-19/V75/axevalla
0 2020-07-25/V75/bollnas


,avd,bins,datum,dist,h1_dist,h1_odds,h1_plac,h1_spår,h2_dist,h2_odds,...,h5_dist,h5_odds,h5_plac,h5_spår,häst,podds,spår,start,vann,vodds
262,4,16,2020-05-09,2640.0,1640,29.36,6.0,4,1609,27.80,...,2140,9.13,5.0,4,NOBLE SUPERB,3.615,6,1,True,17.70
17,2,21,2020-04-25,2640.0,1640,9.32,2.0,3,2140,2.72,...,1600,1.70,1.0,3,VICTORY'S CLUB,6.745,6,0,False,30.97
863,3,15,2020-06-21,2140.0,2160,7.37,1.0,4,2160,10.75,...,1640,2.94,1.0,4,SHARP DREAM,2.580,2,1,False,15.95
193,4,26,2020-04-18,2640.0,2140,3.29,3.0,3,2140,4.33,...,1640,2.25,1.0,3,ANCHOR MAN,7.995,12,0,False,53.98
942,4,19,2020-07-11,2160.0,3180,4.01,1.0,5,1640,51.33,...,2140,10.03,14.0,5,BELLFAKS,3.875,4,1,False,24.35
1042,7,28,2020-07-19,2640.0,2640,50.31,2.0,10,1609,52.58,...,2140,82.70,5.0,10,SIDESLIP,14.055,11,0,False,73.27
782,2,14,2020-07-04,3140.0,2140,2.37,4.0,6,2640,12.18,...,2140,1.40,1.0,4,TWIGS GOT U.,3.735,2,0,False,14.53
204,6,21,2020-04-18,2140.0,2140,2.89,1.0,7,2140,27.90,...,2160,2.84,1.0,7,SHOWTIMEJACKFLOWER,5.465,3,0,False,30.52
479,2,22,2020-05-30,2140.0,2000,5.10,5.0,0,1640,14.18,...,2140,1.68,1.0,0,FRODO S.,5.390,9,0,False,32.82
499,4,12,2020-05-30,2160.0,2140,3.64,6.0,3,2140,9.19,...,2160,4.42,1.0,3,WAITING HILL HALL,3.560,1,0,False,11.69


### för varje modell - räkna ut vinst på df2 (med limits satta ovan)

In [246]:

mod = df2[['datum','avd','häst','vann','vodds']]
#mod = df2.loc[df2.datum==OMGÅNGAR[1].split('/V75')[0],['datum','avd','häst','vann','vodds']]
for i,m in MODELS.items():
    MODELS[i]['res'] = MODELS[i]['model'].result(df.drop('vann',axis=1),i)   
    mod = pd.concat([mod,MODELS[i]['res']], axis=1, sort=False)

df4=mod.copy()
print(df4.shape)
mod_res=['Cat1h_förv','Cat2h_förv','Cat2u_förv','rf301_förv','rf302_förv','knn2_förv']

for res in mod_res:
    print(res)
    df4[res] = df4[res] / df4.vodds

    df4['kelly'] = ((df4.vodds-1) * df4[res] - (1-df4[res])) / (df4.vodds-1) * 100
    
    max=-9999
    the_insats = 0
    the_thresh=2000
    #print(df4.kelly.describe())
    for thresh in range(0,3000):
        utdelning = sum(df4[(df4.vann==1)&(df4.kelly>=thresh)].vodds)
        insats = sum(df4.kelly>thresh)
        if insats == 0:
            break
    
        if utdelning-insats > max:
            max = utdelning-insats 
            the_thresh = thresh
            the_insats = insats
        # if utdelning-insats >0:
        #     print('vinst =',utdelning-insats, 'om thresh = ',thresh )

    
    print('max',max,'med insats =',the_insats,'för thresh =',theThresh)
    # print(df4.kelly.describe())

(1115, 29)
Cat1h_förv
max -1 med insats = 1 för thresh = 27
Cat2h_förv
max -1 med insats = 1 för thresh = 27
Cat2u_förv
max -1 med insats = 1 för thresh = 27
rf301_förv
max -1 med insats = 1 för thresh = 27
rf302_förv
max -1 med insats = 1 för thresh = 27
knn2_förv
max 6.210000000000001 med insats = 11 för thresh = 27


In [247]:
df4=mod.copy()
res = 'knn2_förv'
df4[res] = df4[res] / df4.vodds

df4['kelly'] = ((df4.vodds-1) * df4[res] - (1-df4[res])) / (df4.vodds-1) * 100
max=-9999
#print(df4.kelly.describe())
thresh=27
utdelning = sum(df4[(df4.vann==1)&(df4.kelly>=thresh)].vodds)
print('utd vid thresh',thresh,utdelning)
insats = sum(df4.kelly>thresh)
print('insats',insats)
print('vinst',utdelning-insats )

utd vid thresh 27 7.5
insats 7
vinst 0.5
